In [7]:
import sys
import os
sys.path.append(os.path.abspath("/home/work/disk1/LLM-ljw/agent/analysis-agent"))

In [8]:
import pandas as pd
import numpy as np
from typing import List, Optional
import json
from utils.utils import remove_think


In [9]:
def get_season(month: int) -> str:
    return {12: "Winter", 1: "Winter", 2: "Winter",
            3: "Spring", 4: "Spring", 5: "Spring",
            6: "Summer", 7: "Summer", 8: "Summer",
            9: "Autumn", 10: "Autumn", 11: "Autumn"}[month]

In [10]:
def get_wind_direction_label(degree: float) -> str:
    """将风向角度转换为中文名称（严格匹配气象标准，简洁版）"""
    return ("北", "东北", "东", "东南", "南", "西南", "西", "西北")[
        int(((degree % 360) + 22.5) // 45) % 8
    ]

In [11]:
df = pd.read_excel("../data/test.xlsx")
df

,城市,城市编号,区县,区县编号,站点,唯一编码,时间,风速(m/s),风向(°),气压(hPa),气温(℃),湿度(%),降水量(mm)
0,鄂尔多斯市,150600,达拉特旗,150621,双骏园林,150621001,2024-01-01,0,0,909.5,-8.2,75,0
1,鄂尔多斯市,150600,达拉特旗,150621,双骏园林,150621001,2024-01-02,0,0,910.2,-7.2,69,0
2,鄂尔多斯市,150600,达拉特旗,150621,双骏园林,150621001,2024-01-03,0.2,0,905.9,-7.8,69,0
3,鄂尔多斯市,150600,达拉特旗,150621,双骏园林,150621001,2024-01-04,0,0,904.3,-6.5,63,0
4,鄂尔多斯市,150600,达拉特旗,150621,双骏园林,150621001,2024-01-05,0.1,0,908.2,-7.3,54,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5485,鄂尔多斯市,150600,准格尔旗,150622,气象局,150622002,2024-12-27,2.1,150.2,892,-10.8,46,—
5486,鄂尔多斯市,150600,准格尔旗,150622,气象局,150622002,2024-12-28,0.2,83.5,885.3,-8.9,44,—
5487,鄂尔多斯市,150600,准格尔旗,150622,气象局,150622002,2024-12-29,0.4,94.6,878.9,-5.7,42,—
5488,鄂尔多斯市,150600,准格尔旗,150622,气象局,150622002,2024-12-30,0.7,152.7,882.1,-6.2,39,—


In [12]:
from model_provider.model_provider import MODEL_PROVIDER

In [13]:
from agents import Agent, Runner
from typing import Dict

async def get_column_mapping(df: pd.DataFrame, user_keys: List[str]) -> Dict[str, Optional[str]]:
    """
    将用户提供的变量名（如'风速'）映射到df中的真实列名（如'风速(m/s)'）
    """
    column_mapping_agent = Agent(
        name="列名映射agent",
        instructions="""
        请根据下列 DataFrame 列名，判断哪些列名最接近用户提供的变量意图。
        示例：
        【DataFrame列名】:
        ["风速(m/s)", "风向(16方位)", "PM2.5浓度"]

        【用户意图】:
        ["风速", "风向", "PM2.5", "季节"]

        请输出一个JSON对象，表示用户列名与真实列名的映射关系，如：
        {
        "风速": "风速(m/s)",
        "风向": "风向(16方位)",
        "PM2.5": "PM2.5浓度",
        "季节": None
        }
        请严格遵守以下规则：
        1. 请直接输出JSON对象，不要输出任何其他内容。
        2. 如果存在任何一个列名你认为无法匹配，请设置为None。
        """,
        model=MODEL_PROVIDER.get_model('qwen3:1.7b')
    )
    input = f'DataFrame列名：{df.columns.tolist()}\n用户意图：{user_keys}'
    column_map_result = await Runner.run(
        starting_agent=column_mapping_agent,
        input=input
    )

    return column_map_result

In [ ]:
from agents import Agent
from agents.mcp.server import MCPServerSse,MCPServerSseParams

mcp_param = MCPServerSseParams(
    url='http://127.0.0.1:8000/sse'
)
mcp = MCPServerSse(
    name='corr',
    params=mcp_param,
    client_session_timeout_seconds=180
)
await mcp.connect()

read_timeout_seconds: 0:03:00


Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


In [59]:
from agents.agent import ToolsToFinalOutputFunction, ToolsToFinalOutputResult
from agents import RunContextWrapper
async def async_stop_if_markdown_present(context: RunContextWrapper, tool_results):
    print(tool_results)
    for result in tool_results:
        if isinstance(result, dict) and "markdown" in result:
            return ToolsToFinalOutputResult(
                is_final_output=True,
                final_output=result['markdown']
            )
    return ToolsToFinalOutputResult(
        is_final_output=False,
        final_output=None
    )

In [60]:
from model_provider.model_provider import MODEL_PROVIDER
from agents import Runner
from agents.agent import StopAtTools
agent = Agent(
    name='asistant',
    instructions='你是一个乐于助人的助手。语言：简体中文',
    model=MODEL_PROVIDER.get_model(None),
    mcp_servers=[mcp],
    tool_use_behavior=StopAtTools(stop_at_tool_names=['correlation_analysis'])
)

In [61]:
res = await Runner.run(
    starting_agent=agent,
    input='对./data/corr.csv计算[纳日松]站点的不同季节不同风向方位（不是角度）下的臭氧浓度和风速的相关性',
)

In [62]:
res.to_input_list()

[{'content': '对./data/corr.csv计算[纳日松]站点的不同季节不同风向方位（不是角度）下的臭氧浓度和风速的相关性',
  'role': 'user'},
 {'arguments': '{"correlation_vars":["臭氧浓度","风速"],"filters":{"站点":"纳日松"},"group_by":["季节","风向方位"],"read_data_param":{"read_data_method":"PANDAS","read_data_query":"./data/corr.csv"}}',
  'call_id': 'call_ic1nqgso',
  'name': 'correlation_analysis',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'call_ic1nqgso',
  'output': '{"type":"text","text":"{\\n  \\"result\\": {\\n    \\"冬 - 东\\": 0.473,\\n    \\"冬 - 东北\\": 0.096,\\n    \\"冬 - 东南\\": -0.54,\\n    \\"冬 - 北\\": 0.497,\\n    \\"冬 - 南\\": -100,\\n    \\"冬 - 西\\": 0.657,\\n    \\"冬 - 西北\\": 0.597,\\n    \\"冬 - 西南\\": -100,\\n    \\"夏 - 东\\": -0.559,\\n    \\"夏 - 东北\\": -0.461,\\n    \\"夏 - 东南\\": -0.249,\\n    \\"夏 - 北\\": 0.357,\\n    \\"夏 - 南\\": -0.599,\\n    \\"夏 - 西\\": -0.523,\\n    \\"夏 - 西北\\": -0.397,\\n    \\"夏 - 西南\\": -0.434,\\n    \\"春 - 东\\": -0.095,\\n    \\"春 - 东北\\": -0.225,\\n    \\"春 - 东南\\": -0.999,\\n    \\"

In [63]:
from IPython.display import display, Code, Markdown
display(Markdown(json.loads(json.loads(res.final_output)['text'])['markdown']))

| 分组 | 相关性 |
|---|---|
| 冬 - 东 | 0.473 |
| 冬 - 东北 | 0.096 |
| 冬 - 东南 | -0.54 |
| 冬 - 北 | 0.497 |
| 冬 - 南 | 数据不足 |
| 冬 - 西 | 0.657 |
| 冬 - 西北 | 0.597 |
| 冬 - 西南 | 数据不足 |
| 夏 - 东 | -0.559 |
| 夏 - 东北 | -0.461 |
| 夏 - 东南 | -0.249 |
| 夏 - 北 | 0.357 |
| 夏 - 南 | -0.599 |
| 夏 - 西 | -0.523 |
| 夏 - 西北 | -0.397 |
| 夏 - 西南 | -0.434 |
| 春 - 东 | -0.095 |
| 春 - 东北 | -0.225 |
| 春 - 东南 | -0.999 |
| 春 - 北 | 0.141 |
| 春 - 南 | 数据不足 |
| 春 - 西 | 0.565 |
| 春 - 西北 | 0.451 |
| 秋 - 东 | 0.299 |
| 秋 - 东北 | 0.668 |
| 秋 - 东南 | 0.596 |
| 秋 - 北 | 0.542 |
| 秋 - 南 | 数据不足 |
| 秋 - 西 | 0.421 |
| 秋 - 西北 | -0.003 |


In [145]:
import pandas as pd
tt = pd.read_csv('/home/work/disk1/LLM-ljw/agent/analysis-agent/data/corr.csv',parse_dates=['time'])
tt['time']

0       2024-01-01
1       2024-01-01
2       2024-01-01
3       2024-01-01
4       2024-01-01
           ...    
10749   2024-12-31
10750   2024-12-31
10751   2024-12-31
10752   2024-12-31
10753   2024-12-31
Name: time, Length: 10754, dtype: datetime64[ns]

In [163]:
tt = tt[tt['站点名称'] == '纳日松']

In [164]:
tt.columns

Index(['time', '站点名称', 'O3_8H', '气温(℃)', '风速(m/s)', '风向(°)', 'SO2', 'NO2',
       'PM10', 'CO', 'PM2.5', 'season', 'wind_direction'],
      dtype='object')

In [165]:
tt['season'] = tt['time'].dt.month.map(get_season)
tt['wind_direction'] = tt['风向(°)'].apply(get_wind_direction_label)

/tmp/ipykernel_537143/2181503513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt['season'] = tt['time'].dt.month.map(get_season)
/tmp/ipykernel_537143/2181503513.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt['wind_direction'] = tt['风向(°)'].apply(get_wind_direction_label)


In [166]:
g = tt.groupby(['season','wind_direction'])

In [167]:
sub = g[['O3_8H','风速(m/s)']]

In [168]:
grouped = tt.groupby(['season','wind_direction'])
var1 = 'O3_8H'
var2 = '风速(m/s)'
result = {}
for keys, group in grouped:
    sub = group[[var1, var2]].dropna()
    key_str = " | ".join(str(k) for k in keys) if isinstance(keys, tuple) else str(keys)
    if sub.shape[0] < 3:
        corr = -100
        result[key_str] = corr
    else:
        corr = sub[var1].corr(sub[var2])
        result[key_str] = corr

In [169]:
result

{'Autumn | 东': 0.29944683692745494,
 'Autumn | 东北': 0.6678641128818839,
 'Autumn | 东南': 0.5960812289978803,
 'Autumn | 北': 0.5420985524564963,
 'Autumn | 南': -100,
 'Autumn | 西': 0.4206467933361614,
 'Autumn | 西北': -0.0032086849347231963,
 'Spring | 东': -0.09537936202206765,
 'Spring | 东北': -0.2254869352625949,
 'Spring | 东南': -0.9989400229922533,
 'Spring | 北': 0.14111818158906386,
 'Spring | 南': -100,
 'Spring | 西': 0.5651595452693585,
 'Spring | 西北': 0.4513850668355761,
 'Summer | 东': -0.5586814731036658,
 'Summer | 东北': -0.46066523675557797,
 'Summer | 东南': -0.248762631000327,
 'Summer | 北': 0.3569839681011098,
 'Summer | 南': -0.5991770741594884,
 'Summer | 西': -0.522570348896076,
 'Summer | 西北': -0.39699987493325833,
 'Summer | 西南': -0.4344989263168032,
 'Winter | 东': 0.4729844478671217,
 'Winter | 东北': 0.09562507962407665,
 'Winter | 东南': -0.5400663288733786,
 'Winter | 北': 0.49677957769177533,
 'Winter | 南': -100,
 'Winter | 西': 0.6565488767470031,
 'Winter | 西北': 0.597244095514

In [157]:
sub['O3_8H'].corr(sub['风速(m/s)'])

IndexError: Column(s) ['O3_8H', '风速(m/s)'] already selected